<a href="https://colab.research.google.com/github/dayana95/dayana95.github.io/blob/main/RSO_para_VRP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import random as rnd
import pandas as pd
import numpy as np
import time

class Problem:
    def __init__(self):
        self.dimension = 11
        self.clients = range(self.dimension)
        self.costs = [
            [0,  5,  10, 15, 7,  12, 20, 8,  14, 10, 9],
            [5,  0,  6,  9,  5,  8,  14, 4,  7,  5,  6],
            [10, 6,  0,  8,  4,  6,  13, 7,  5,  8,  9],
            [15, 9,  8,  0,  7,  4,  10, 5,  3,  6,  12],
            [7,  5,  4,  7,  0,  3,  9,  6,  4,  5,  8],
            [12, 8,  6,  4,  3,  0,  7,  3,  5,  6,  9],
            [20, 14, 13, 10, 9,  7,  0,  6,  5,  8,  10],
            [8,  4,  7,  5,  6,  3,  6,  0,  2,  5,  7],
            [14, 7,  5,  3,  4,  5,  5,  2,  0,  3,  8],
            [10, 5,  8,  6,  5,  6,  8,  5,  3,  0,  4],
            [9,  6,  9,  12, 8,  9,  10, 7,  8,  4,  0]
        ]

    def check(self, s):
        table = []
        for i in self.clients:
            if s[i] in table:
                return False
            table.append(s[i])

        visited = []
        next = self.clients[0]
        for _ in self.clients:
            pos = s[next]
            if pos in visited:
                return False
            visited.append(pos)
            next = pos
        return True

    def fit(self, s):
        cost = 0
        for i in self.clients:
            cost += self.costs[i][s[i]]
        return cost

class Rat:
    def __init__(self):
        self.p = Problem()
        table = list(self.p.clients)
        self.pos = [table.pop(rnd.randint(0, len(table) - 1)) for _ in self.p.clients]

    def isfeasible(self):
        return self.p.check(self.pos)

    def isbetterthan(self, o):
        return self.fitness() < o.fitness()

    def fitness(self):
        return self.p.fit(self.pos)

    def move(self, g, A, C):
        for j in self.p.clients:
            self.pos[j] = (A * self.pos[j]) + (C * (g.pos[j] - self.pos[j]))
            self.pos[j] = abs(g.pos[j] - self.pos[j])

        lista = sorted(self.pos)
        self.pos = [lista.index(pos) for pos in self.pos]

    def __str__(self) -> str:
        return f"fit:{self.fitness()} x:{self.pos}"

    def copy(self, a):
        self.pos = a.pos.copy()

class RSO:
    def __init__(self, maxiter, nrats):
        self.maxiter = maxiter
        self.nrats = nrats
        self.swarm = []
        self.Rmin = 1
        self.Rmax = 5
        self.cValue = 2.0
        self.g = Rat()

    def solve(self):
        self.initrand()
        self.evolve()
        return self.g.fitness()

    def initrand(self):
        self.swarm = list()
        for i in range(self.nrats):
            while True:
                p = Rat()
                if p.isfeasible():
                    break
            self.swarm.append(p)

        self.g.copy(self.swarm[0])
        for i in range(1, self.nrats):
            if self.swarm[i].isbetterthan(self.g):
                self.g.copy(self.swarm[i])

    def evolve(self):
        t = 1
        while t <= self.maxiter:
            for i in range(self.nrats):
                while True:
                    testRat = Rat()
                    testRat.copy(self.swarm[i])
                    C = self.cValue * rnd.random()
                    R = rnd.uniform(self.Rmin, self.Rmax)
                    A = R - t*(R / self.maxiter)
                    testRat.move(self.g, A, C)
                    if testRat.isfeasible():
                        self.swarm[i].copy(testRat)
                        del testRat
                        break

                if self.swarm[i].isbetterthan(self.g):
                    self.g.copy(self.swarm[i])
            t += 1
